<a href="https://colab.research.google.com/github/sakshiigdtuw/ibm-repo/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas scikit-learn nltk flask joblib

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Schemes.zip to Schemes.zip


In [ ]:
import os

# List files in the current directory
uploaded_files = os.listdir('/content/')
print("Files in /content/:", uploaded_files)


Files in /content/: ['.config', 'Schemes.zip', 'sample_data']


In [ ]:
import zipfile
import os

# Define the filename of the zip file
zip_file_name = 'Schemes.zip'  # Use the exact name from the directory listing

# Define the extraction path
extraction_path = '/content/schemes_extracted'  # Directory to extract files

# Create the extraction directory if it doesn't exist
os.makedirs(extraction_path, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)
    print(f"Extracted {zip_file_name} to {extraction_path}")

# List files in the extraction directory
extracted_files = os.listdir(extraction_path)
print("Extracted files:", extracted_files)


Extracted Schemes.zip to /content/schemes_extracted
Extracted files: ['kerala', 'mizoram', 'jammu-kashmir', 'karnataka', 'chhattisgarh', 'maharashtra', 'bihar', 'madhya-pradesh', 'meghalaya', 'manipur', 'rajasthan', 'tamilnadu', 'punjab', 'tripura', 'west-bengal', 'gujarat', 'assam', 'goa', 'uttar-pradesh', 'chandigarh', 'odisha', 'central', 'himachal-pradesh', 'haryana', 'uttarakhand', 'delhi', 'jharkhand', 'andhra-pradesh', 'telangana']


In [ ]:
# List all files and directories in the extraction directory
for root, dirs, files in os.walk(extraction_path):
    for name in files:
        print(os.path.join(root, name))
    for name in dirs:
        print(os.path.join(root, name))


/content/schemes_extracted/kerala
/content/schemes_extracted/mizoram
/content/schemes_extracted/jammu-kashmir
/content/schemes_extracted/karnataka
/content/schemes_extracted/chhattisgarh
/content/schemes_extracted/maharashtra
/content/schemes_extracted/bihar
/content/schemes_extracted/madhya-pradesh
/content/schemes_extracted/meghalaya
/content/schemes_extracted/manipur
/content/schemes_extracted/rajasthan
/content/schemes_extracted/tamilnadu
/content/schemes_extracted/punjab
/content/schemes_extracted/tripura
/content/schemes_extracted/west-bengal
/content/schemes_extracted/gujarat
/content/schemes_extracted/assam
/content/schemes_extracted/goa
/content/schemes_extracted/uttar-pradesh
/content/schemes_extracted/chandigarh
/content/schemes_extracted/odisha
/content/schemes_extracted/central
/content/schemes_extracted/himachal-pradesh
/content/schemes_extracted/haryana
/content/schemes_extracted/uttarakhand
/content/schemes_extracted/delhi
/content/schemes_extracted/jharkhand
/content/s

In [ ]:
import os

# Path to the extracted dataset
dataset_folder_path = '/content/schemes_extracted'

def load_data_from_folder(folder_path):
    data = []
    labels = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                data.append(content)
                # Optionally, you could use folder names or file names as labels
                # Here we use the folder name as the label
                labels.append(os.path.basename(root))
    return data, labels

# Load the data
schemes_data, labels = load_data_from_folder(dataset_folder_path)

# Print the first entry for verification
print(schemes_data[0])
print(labels[0])



Kerala govt. has proposed a new Health Policy 2018 in which govt. will make vaccination cards mandatory for students seeking admission in 1st standard. Subsequently, parents must get their child vaccinated for admissions in school which will reduce the Mortality Rate of children. Moreover, Kerala Health policy 2018 also contains provision of strict legal action against people who discourage parents from getting their children vaccinated. 
The state govt. has planned this move after several vaccination programmes facing resistance in Kerala. Most recent example is the Measles Rubella (MR) vaccination drive in few districts. In addition to this, govt. will also make standard treatment guidelines compulsory for doctors at govt. hospitals and also at private hospitals.
Subsequently, Kerala govt. will also take efficient steps to curb corruption and strengthen vigilance system in hospitals. Moreover, this policy will ensure that all the children below the age of 5 are immunized and will ac

In [ ]:
import re

def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r'\d+', '', text)   # Remove numbers
    text = text.lower()               # Convert to lowercase
    return text

schemes_data = [preprocess_text(entry) for entry in schemes_data]


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
import pickle

# Vectorize the text data
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(schemes_data)

# Train a KNN classifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X, labels)

# Save the model and vectorizer
with open('/content/vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)
with open('/content/knn_model.pkl', 'wb') as f:
    pickle.dump(knn, f)


In [ ]:
from flask import Flask, request, jsonify
import pickle

app = Flask(__name__)

# Load the model and vectorizer
with open('/content/vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)
with open('/content/knn_model.pkl', 'rb') as f:
    knn = pickle.load(f)

@app.route('/chat', methods=['POST'])
def chat():
    user_input = request.json.get('message')
    prediction = get_scheme_from_text(user_input)
    return jsonify({'response': prediction})

def get_scheme_from_text(text):
    text_vector = vectorizer.transform([text])
    prediction = knn.predict(text_vector)
    return prediction[0]

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
